In [1]:
import bluebook

In [2]:
bluebook.list_models()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (

In [3]:
bluebook.list_gpus()

['NVIDIA GeForce RTX 2060 SUPER']

In [4]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [5]:
import json
import syft as sy
sy.requires(SYFT_VERSION)

/home/teo/OpenMined/PySyft/packages/syft/src/syft/gevent_patch.py:42: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/site-packages/urllib3/util/__init__.py)', 'urllib3.util.ssl_ (/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/site-packages/urllib3/util/ssl_.py)', 'urllib3.contrib.pyopenssl (/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/site-packages/urllib3/contrib/pyopenssl.py)', 'botocore.httpsession (/home/teo/anaconda3/envs/syft_3.11/lib/python3.11/site-packages/botocore/httpsession.py)']. 
  monkey.patch_all(thread=False)
kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/te

✅ The installed version of syft==0.8.2b6 matches the requirement >=0.8.2b0 and the requirement <0.9


In [6]:
node = sy.orchestra.launch(name="blue-book", port="auto", dev_mode=True, reset=True)

Starting blue-book server on 0.0.0.0:14103

Waiting for server to start

INFO:     Started server process [1524999]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:14103 (Press CTRL+C to quit)


INFO:     127.0.0.1:60676 - "GET /api/v2/metadata HTTP/1.1" 200 OK
. Done.


In [7]:
domain_client = node.login(email="info@openmined.org", password="changethis")

SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

INFO:     127.0.0.1:60678 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:60678 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:60678 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:60678 - "GET /api/v2/types?verify_key=8ca7d1f9b2967b2b2c6155cd51e61ae8617e51e5e2658ab27af27faab51e6be6 HTTP/1.1" 200 OK
INFO:     127.0.0.1:60678 - "GET /api/v2/api?verify_key=8ca7d1f9b2967b2b2c6155cd51e61ae8617e51e5e2658ab27af27faab51e6be6 HTTP/1.1" 200 OK


Logged into <blue-book: High side Domain> as <info@openmined.org>


In [8]:
from syft.serde.lib_service_registry import CMPModule, CMPTree, ALL_EXECUTE, CMPBase, CMPFunction, CMPClass
bluebook_cmp = CMPTree(
    children=[CMPModule(
    "bluebook",
    permissions=ALL_EXECUTE,
    )]
).build()

In [9]:
bluebook_cmp.bluebook

├───bluebook (ALL_EXECUTE)
│    ├───example (ALL_EXECUTE)
│    │    ├───list_gpus (ALL_EXECUTE)
│    │    └───list_models (ALL_EXECUTE)
│    ├───list_gpus (ALL_EXECUTE)
│    └───list_models (ALL_EXECUTE)

In [11]:
from syft.service.service import register_lib_obj, action_execute_registry_libs, LibConfigRegistry

# hacky, prevent circular imports
for lib_obj in bluebook_cmp.flatten():
    # # for functions
    # func_name = func.__name__
    # # for classes
    # func_name = path.split(".")[-1]
    if isinstance(lib_obj, CMPFunction) or isinstance(lib_obj, CMPClass):
        print(lib_obj)
        register_lib_obj(lib_obj)

├───bluebook.example.list_gpus (ALL_EXECUTE)

├───bluebook.example.list_models (ALL_EXECUTE)

├───bluebook.list_gpus (ALL_EXECUTE)

├───bluebook.list_models (ALL_EXECUTE)



In [14]:
sy.serialize(bluebook_cmp)

Exception: builtins.module not in TYPE_BANK

In [12]:
list(LibConfigRegistry.__service_config_registry__.keys())

['bluebook.example.list_gpus',
 'bluebook.example.list_models',
 'bluebook.list_gpus',
 'bluebook.list_models']

In [13]:
module = CMPModule(
    "bluebook",
    permissions=ALL_EXECUTE,
    )
domain_client.api.services.python_lib.add_lib(cmp_module=bluebook_cmp)

Exception: builtins.module not in TYPE_BANK

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <blue-book: High side Domain> as <info@openmined.org>


In [ ]:
domain_client.api.services.python_lib.show_lib()

[]

In [ ]:
domain_client.api.lib.bluebook

---------------------------------------------------------------------------
SyftAttributeError
---------------------------------------------------------------------------
Exception: 'APIModule' api object has no submodule or method 'bluebook', you may not have permission to access the module you are trying to access


In [ ]:
python_lib_cli_dockerfile = """
FROM nvidia/cuda:11.4.0-base-ubuntu20.04

# Set environment variables
ENV DEBIAN_FRONTEND=noninteractive

# Install system dependencies
RUN apt-get update && \
    apt-get install -y \
        git \
        python3-pip \
        python3-dev \
        python3-opencv \
        libglib2.0-0

RUN python3 -m pip install -r requirements.txt

# Upgrade pip
RUN python3 -m pip install --upgrade pip

# Install PyTorch and torchvision
RUN pip3 install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html


# Set the entrypoint
ENTRYPOINT [ "python3" ]
"""


In [ ]:
result = domain_client.api.services.container.add_image(
    name="python_lib",
    tag="python_lib:latest",
    dockerfile=python_lib_cli_dockerfile,
    # volumes=volumes,
)
result

SyftSuccess: ContainerImage added: syft.service.container.container.ContainerImage

In [ ]:
result = domain_client.api.services.container.get_images()
result

[syft.service.container.container.ContainerImage]